In [42]:
import sys
sys.path.append("../../")
import PolyMID
import pandas as pd
from PolyMID.AnalyzeSpectra import fragment_library
import numpy as np

from pdb import set_trace

In [2]:
file_directory = './'
metabolite_dict,metabolite_list = fragment_library.fragment_library(file_directory=file_directory,Full_NC=False)

    pyruvate_mox_tbdms
    lactate_2tbdms
    alanine_2tbdms
    glycine_2tbdms
    3-hydroxypropionicacid_2tbdms
    valine_2tbdms
    norvaline_2tbdms
    beta-alanine_2tbdms
    leucine_2tbdms
    isoleucine_2tbdms
    succinate_2tbdms
    uracil_2tbdms
    fumarate_2tbdms
    serine_3tbdms
    threonine_3tbdms
    taurine_2tbdms
    methionine_2tbdms
    alpha-ketoglutarate_mox_2tbdms
    malate_3tbdms
    phenylalanine_2tbdms
    aspartate_3tbdms
    proline_MOX_2tbdms
    glutamate_3tbdms
    asparagine_3tbdms
    lysine_3tbdms
    glutamine_3tbdms
    arginine_4tbdms
    citrate_4tbdms
    tyrosine_3tbdms
    histidine_3tbdms
    tryptophan_3tbdms
    glucose
    glucose_clean


In [34]:
Standard_MID_DF = pd.read_csv('Internal_Standards.txt', delimiter='\t', index_col=0)
fragments = Standard_MID_DF.columns

for met_key in metabolite_dict.keys():
    for frag_key in metabolite_dict[met_key]['fragments'].keys():
        if frag_key in fragments:
            Natural_MID = metabolite_dict[met_key]['fragments'][frag_key]['natural_mid'][0:13]
            frag_key_position = Standard_MID_DF.columns.get_loc(frag_key)
            Standard_MID_DF.iloc[0:13,frag_key_position] = Natural_MID
            Standard_MID_DF.iloc[13:26,frag_key_position] = np.zeros(13)

Standard_MID_DF.to_csv('Internal_Standards_Filled.txt', sep='\t', index=True, header=True)

In [47]:
Labeled_Standard_DF = pd.read_csv('Internal_Standards_Uncorrected.txt', delimiter='\t', index_col=0)

MID_indices = [False]*len(Labeled_Standard_DF.index)
for i in range(len(Labeled_Standard_DF.index)):
    MID_Condition = (str(Labeled_Standard_DF.index[i])[-2] == 'M') | (str(Labeled_Standard_DF.index[i])[-3] == 'M')
    MID_indices[i] = MID_Condition

MID_DF = Labeled_Standard_DF[MID_indices].copy()
MID_DF.index = [MID_DF.index[i][0:-3] for i in range(len(MID_DF.index))]
MID_DF.drop('predicted',axis=1,inplace=True)
MID_DF['average'] = MID_DF.iloc[:, 0:3].mean(axis=1)
print(MID_DF)

                 is01      is02  is03   average
lactate_233  0.742905  0.741679   0.0  0.494861
lactate_233  0.158084  0.159010   0.0  0.105698
lactate_233  0.078177  0.079529   0.0  0.052569
lactate_233  0.012595  0.011241   0.0  0.007945
lactate_233  0.008240  0.008541   0.0  0.005594
...               ...       ...   ...       ...
glucose_273  0.000000  0.000000   0.0  0.000000
glucose_273  0.000000  0.000000   0.0  0.000000
glucose_273  0.000000  0.000000   0.0  0.000000
glucose_273  0.000000  0.000000   0.0  0.000000
glucose_273  0.000000  0.000000   0.0  0.000000

[526 rows x 4 columns]


In [48]:
for frag in fragments:
    frag_indices = MID_DF.index ==frag
    frag_MID = MID_DF.loc[frag_indices,'average']
    if frag=='phenylalanine_336':
        set_trace()

    frag_position = Standard_MID_DF.columns.get_loc(frag)
    Standard_MID_DF.iloc[13:(13+len(frag_MID)),frag_position] = frag_MID
    

> /var/folders/mz/w7vwcsb17098wvqdr84by1sc0000gr/T/ipykernel_16462/2835337949.py(7)<module>()
      5         set_trace()
      6 
----> 7     frag_position = Standard_MID_DF.columns.get_loc(frag)
      8     Standard_MID_DF.iloc[13:(13+len(frag_MID)),frag_position] = frag_MID
      9 

phenylalanine_336    0.000000
phenylalanine_336    0.000000
phenylalanine_336    0.000000
phenylalanine_336    0.000000
phenylalanine_336    0.000000
phenylalanine_336    0.000257
phenylalanine_336    0.000478
phenylalanine_336    0.001384
phenylalanine_336    0.002993
phenylalanine_336    0.075575
Name: average, dtype: float64


In [41]:
Standard_MID_DF.to_csv('Internal_Standards_Filled.txt', sep='\t', index=True, header=True)